For some reason, I wanted to take the data in 'main' and add things to it, to make 'parsed'.  
This was in June 2024, ish

In [1]:
import pandas as pds
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# load in data from 'main' spreadsheet
main = pd.read_csv('main.csv')
print(len(main))
main.head(3)

180


,Gene Symbol,Gene Name,Disease Association,Human Protein ID,Top Tardigrade,Disease Family
0,ANO5,anoctamin-5,Miyoshi myopathy,NP_001397892,GBZR01001446,Muscular
1,ATP13A2,polyamine-transporting ATPase 13A2,Parkinson’s disease,NP_071372,GBZR01001442,Degenerative (brain)
2,UBQLN2,ubiquilin-1,Amyotrophic lateral sclerosis,NP_444295,GBZR01000959,Degenerative (motor neuron)


In [3]:
# get lists and remove trailing whitespace
gene_symbols = main['Gene Symbol'].values
gene_names = main['Gene Name'].values
disease_associations = main['Disease Association'].values
human_prot = main['Human Protein ID'].values
tardigrade_hit = main['Top Tardigrade'].values
disease_family = main['Disease Family'].values

for li in [gene_symbols, gene_names, disease_associations, human_prot, tardigrade_hit, disease_family]:
    for i,x in enumerate(li):
        try:
            y=x.rstrip()
        except:
            y=x
        li[i] = y
    
print(gene_symbols[:5])
print(disease_family[:5])

['ANO5' 'ATP13A2' 'UBQLN2' 'LDB3' 'ACTA1']
['Muscular' 'Degenerative (brain)' 'Degenerative (motor neuron)'
 'Muscular' 'Muscular']


In [4]:
# make dictionaries of stuff that I want to use later
names = dict(zip(gene_symbols, gene_names))
diseases = dict(zip(gene_symbols, disease_associations))
disease_categories = dict(zip(gene_symbols, disease_family))
tardigrade_matches = dict(zip(gene_symbols, tardigrade_hit))
human_accessions = dict(zip(gene_symbols, human_prot))

In [5]:
# schlurp up sites_and_papers data
f = open('sites_papers.csv','r')
data = f.readlines()
f.close()
print(len(data))
print(data[:3])

364
['MDSgene,Spinocerebellar Ataxia (ATX),TBP,\n', 'MDSgene,Chorea,ADCY5,\n', 'MDSgene,Chorea,NKX2-1,\n']


In [19]:
# go line by line through sites_and_papers data
lowered_symbols = [x.lower() for x in gene_symbols]
sources = {}

for line in data:
    line = line.rstrip()
    stuff = line.split(',')
    
    source = stuff[0]
    disease = stuff[1]
    symbol = stuff[2]
    family = stuff[3]

    if symbol.lower() in lowered_symbols:
        # we have this gene! add new information to appropriate dictionaries
#         print('We have ' + symbol)
        
        # disease: does this gene already have one?
        if symbol in diseases.keys():
            
            # is this disease already in there?
            try:
                if disease.lower() not in diseases[symbol].lower():
                    diseases[symbol] += '; ' + disease
#                     print('  appended a disease for ' + symbol)
#                     print(diseases[symbol])
            except:
                pass
                
        else:
            print('we have not seen ' + symbol + ' yet')
            diseases[symbol] = disease
            print('   created a disease for ' + symbol)
        
        # disease category: does this gene already have one?
        if symbol in disease_categories.keys():
            
            # yes! is this category already in there?
            try:
                if family.lower() not in disease_categories[symbol].lower():
                    disease_categories[symbol] += '; ' + family
            except:
                pass
        else:
            disease_categories[symbol] = family
            
    # does this gene already have a source?
    if symbol in sources.keys():
#         print("already have a source for " + symbol)
        # yes ... check if this source is already in there
        if source not in sources[symbol]:
            sources[symbol] += '; ' + source

    else:
        sources[symbol] = source
#         print('new source for ' + symbol)

    diseases[symbol] = disease    
    disease_categories[symbol] = family
        
    

In [20]:
print(len(gene_symbols))
print(len(sources))
print(len(diseases))
print(len(disease_categories))

180
245
293
293


In [18]:
print(sources['CACNA1A'])

https://medlineplus.gov/genetics/condition/spinocerebellar-ataxia-type-6/#causes; Cordeiro 2018


In [26]:
gene_symbols = sorted(diseases.keys())
name_col = [''] * len(gene_symbols)
disease_col = [''] * len(gene_symbols)
dis_cats_col = [''] * len(gene_symbols)
tard_col = [''] * len(gene_symbols)
hum_acc_col = [''] * len(gene_symbols)
source_col = [''] * len(gene_symbols)

for i, symbol in enumerate(gene_symbols):
    if symbol in names.keys():
        name_col[i] = names[symbol]
    if symbol in diseases.keys():
        disease_col[i] = diseases[symbol]
    if symbol in disease_categories.keys():
        dis_cats_col[i] = disease_categories[symbol]
    if symbol in tardigrade_matches.keys():
        tard_col[i] = tardigrade_matches[symbol]
    if symbol in human_accessions.keys():
        hum_acc_col[i] = human_accessions[symbol]
    if symbol in sources.keys():
        source_col[i] = sources[symbol]


In [27]:
for li in [gene_symbols, name_col, disease_col, dis_cats_col, source_col, hum_acc_col, tard_col]:
    print(len(li))

293
293
293
293
293
293
293


In [31]:
df = pd.DataFrame({
    'Gene Symbol': gene_symbols,
    'Source':source_col,
    'Gene Name': name_col,
    'Disease': disease_col,
    'Disease Category': dis_cats_col,
    'Human Protein':hum_acc_col,
    'Tardigrade Match':tard_col
})
df.head(30)

,Gene Symbol,Source,Gene Name,Disease,Disease Category,Human Protein,Tardigrade Match
0,,,,,,,
1,ACTA1,,actin: alpha skeletal muscle,Congenital fiber-type disproportion,Muscular,NP_001091,GBZR01012379
2,ACVR1,,activin receptor type-1,Fibrodysplasia ossificans progressiva,Muscular,NP_001096,GBZR01001950
3,ADCY5,MDSgene; https://medlineplus.gov/genetics/cond...,adenylate cyclase type 5,Chorea,,NP_899200,GBZR01004753
4,AGL,,glycogen debranching enzyme,Glycogen storage disease type III,Muscular,NP_000019,GBZR01003968
5,ALDH5A1,https://medlineplus.gov/genetics/condition/suc...,succinate-semialdehyde dehydrogenase: mitochon...,Succinic semialdehyde dehydrogenase deficiency,Neurotransmission,NP_001071,GBZR01011928
6,ALS2,https://medlineplus.gov/genetics/condition/amy...,Amyotrophic lateral sclerosisin,Juvenile primary lateral sclerosis,Degenerative,NP_001129217,GBZR01008132
7,AMACR,https://medlineplus.gov/genetics/condition/alp...,alpha-methylacyl-CoA racemase,Alpha-methylacyl-CoA racemase deficiency,Degenerative,NP_055139,GBZR01003857
8,AMPD1,,AMP deaminase 1,Adenosine monophosphate deaminase deficiency,Muscular,NP_000027,GBZR01012479
9,ANG,https://medlineplus.gov/genetics/condition/amy...,,Amyotrophic Lateral Sclerosis,Degenerative,,


In [32]:
df.to_csv('parsed.csv')